<a href="https://colab.research.google.com/github/Hemanthnaidu3737/Sentiment-Analysis-using-BERT-and-Transformers/blob/main/BERT_SENTIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gradio transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import gradio as gr
from transformers import pipeline
import torch
import time
import matplotlib.pyplot as plt
import numpy as np


In [5]:
# Enable GPU if available
device = 0 if torch.cuda.is_available() else -1

In [6]:
# Intent-based responses
intent_responses = {
    "positive": ["That's great to hear! 😊", "Awesome! Keep spreading positivity! 🎉"],
    "neutral": ["Okay, got it! 😐", "Thanks for sharing!"],
    "negative": ["I'm sorry to hear that. 😔", "Let's try to make things better! 🌟"]
}

In [7]:
# Load pre-trained sentiment analysis model
sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Prediction function
def predict_sentiment(user_input):
    if isinstance(user_input, list):
        results = [sentiment_pipeline(text)[0] for text in user_input]
        sentiments = []

        for res in results:
            label = res["label"]
            if "5 stars" in label or "4 stars" in label:
                sentiments.append("positive")
            elif "3 stars" in label:
                sentiments.append("neutral")
            else:
                sentiments.append("negative")

        # Plotting results
        labels, counts = np.unique(sentiments, return_counts=True)
        fig, ax = plt.subplots()
        ax.bar(labels, counts, color=['green', 'yellow', 'red'])
        ax.set_title("Sentiment Distribution")
        ax.set_ylabel("Count")

        return "Batch analysis complete!", fig

    else:
        time.sleep(1)  # Simulate loading
        result = sentiment_pipeline(user_input)
        label = result[0]["label"]
        score = result[0]["score"]

        if "5 stars" in label or "4 stars" in label:
            sentiment = "positive"
        elif "3 stars" in label:
            sentiment = "neutral"
        else:
            sentiment = "negative"

        response = intent_responses[sentiment][0]

        return f"Sentiment: {sentiment.capitalize()}\nConfidence: {score:.2f}\nResponse: {response}", None

In [9]:
# Custom Theme
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes

class CustomColorfulTheme(Base):
    def __init__(self):
        super().__init__(
            primary_hue=colors.blue,
            secondary_hue=colors.emerald,
            font=fonts.GoogleFont("Poppins"),
            radius_size=sizes.radius_lg,
            spacing_size=sizes.spacing_md,
            text_size=sizes.text_lg
        )

custom_theme = CustomColorfulTheme()

In [10]:
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter your text here..."),
    outputs=["text", "plot"],
    title="💬 Sentiment Analysis Using BERT & Transformers",
    description="Analyze the sentiment of your text using a BERT-based multilingual model.",
    theme=custom_theme  # ← Apply the theme here
)


In [18]:
gr.Markdown("# 🧠 Sentiment Analysis using BERT and Transformers")



In [19]:
gr.Image(value="C:/Users/heman/Downloads/mout.jpg", type="filepath", label="Your Logo", height=150, show_label=False)



In [20]:
import gradio as gr
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes

class ColorfulTheme(Base):
    def __init__(self):
        super().__init__(
            primary_hue=colors.sky,
            secondary_hue=colors.orange,
            font=fonts.GoogleFont("Poppins"),
            spacing_size=sizes.spacing_md,
            radius_size=sizes.radius_lg,
            text_size=sizes.text_md
        )

# Create an instance of the custom theme
custom_theme = ColorfulTheme()


In [21]:
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter your text here..."),
    outputs=["text", "plot"],
    title="💬 Sentiment Analysis Using BERT & Transformers",
    description="Analyze the sentiment of your text using a BERT-based multilingual model.",
    theme=custom_theme  # ← This applies the theme you defined above
)


In [22]:
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a34edd819f6b787f97.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
